In [1]:
SEGMENT_SIZE=2**18
IN_TEMP='temp.data'
IN_COMPRESSED='baby-elephant-walk-1.0.jamm'
OUT_AUDIO='baby-elephant-walk-1.0.wav'

In [2]:
import zipfile
import scipy.io.wavfile
import scipy.fft
import numpy as np
import pickle

def load_compressed_zip():
    with zipfile.ZipFile(IN_COMPRESSED, 'r') as zip:
        with zip.open(IN_TEMP, 'r') as infile:
            text = infile.read().decode("utf-8")
            return text
        
def load_compressed():
    with zipfile.ZipFile(IN_COMPRESSED, 'r') as zip:
        return pickle.load(zip.open(IN_TEMP, 'r'))
        
def write_audio_from_terms_text(text):
    lines = text.split('\n')
    rate = int(lines[0])
    
    terms = []
    for line in lines[1:]:
        if line.startswith('x'):
            count = int(line[1:])
            for i in range(count):
                terms.append(0)
        elif line:
            tokens = line.split(' ')
            terms.append(complex(float(tokens[0]), float(tokens[1])))
    
    inverted = scipy.fft.ifft(terms)
    scipy.io.wavfile.write(OUT_AUDIO, rate, np.array([item / rate for item in inverted], float))
    
    
def write_audio_from_terms(lines):
    rate = lines[0]
    
    terms = []
    for line in lines[1:]:
        if type(line) is int:
            for i in range(line):
                terms.append(0)
        elif len(line) == 2:
            terms.append(complex(line[0], line[1]))
    
    inverted = scipy.fft.ifft(terms)
    scipy.io.wavfile.write(OUT_AUDIO, rate, np.array([item.real / rate for item in inverted], float))
    
write_audio_from_terms(load_compressed())